In [ ]:
import sys

sys.path.append('./src')

%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import os.path
import filters
from datetime import date, datetime, time
import time as pytime
from dataclasses import dataclass
from typing import Optional, Dict
import pandas as pd

from time_block import TimeBlock
from parks_web import get_availability

In [ ]:
blocks = filter(None, map(filters.clip_range_date, get_availability('Mag8', '2025-05-01', '2025-07-30')))
blocks = filters.filter_duration(blocks, 0.9)
list(blocks)[:10]

In [ ]:
df = pd.DataFrame(map(lambda b: b.to_json(), blocks))
#df.sort_values('Date')
#df.to_csv('~/Downloads/lower_woodland.csv')
df

In [ ]:
date(2025, 7, 3) in HOLIDAYS

In [ ]:
get_availability('Mag8', '2025-06-01', '2025-06-30')